# Идея проекта, выборка и предобработка данных
<hr>

## Введение

```Текущая accuracy: 60-70%```

Проект представляет из себя анализатор дипфейков на основе анализа артефактов в различных зонах лица

В ходе предварительного анализа предметной области было выделено 3 наиболее перспективных для реализации метода анализа дипфейков:

- анализ движений глаз (саккад)
- анализ синзронизации речи и движений губ
- анализ артефактов

Так как анализ движений возможен только в динамике и довольно сложен в реализации, было решено реализовать метод анализа артефактов на статических изображениях (фото)

- В качестве "источника вдохновения" использовался [проект](https://github.com/rakshitsakhuja/Detecting-Deepfakes-with-OpenCV/blob/master/1.%20Processing%20Videos%20and%20Face%20Detection.ipynb), доступный по ссылке: 

https://github.com/rakshitsakhuja/Detecting-Deepfakes-with-OpenCV/blob/master/1.%20Processing%20Videos%20and%20Face%20Detection.ipynb

- В качестве датасета использовались фрагменты датасета [Deep Fake Detection Challenge DFDC](https://www.kaggle.com/competitions/deepfake-detection-challenge):

https://www.kaggle.com/competitions/deepfake-detection-challenge

- Про [распознанвание зон лица](https://pyimagesearch.com/2017/04/10/detect-eyes-nose-lips-jaw-dlib-opencv-python/) можно прочесть по ссылке:

https://pyimagesearch.com/2017/04/10/detect-eyes-nose-lips-jaw-dlib-opencv-python/

- Сжатый [датасет 300 Faces In-the-Wild Challenge (300-W)](https://ibug.doc.ic.ac.uk/resources/300-W/) для распознавания точек на лице и аннотации к нему:

https://ibug.doc.ic.ac.uk/resources/300-W/

## Пайплайн проекта

Проект работает в следующем порядке:

- datacollector.py, faceparts.py:
1. Считывание по 1 кадру из видео-датасета DFDC (5 x 10 GB, 5 x 1700 видео)
2. Распознавание 68 точек на лице с помощью OpenCV, DLib и датасета 300-W
3. Отсев фото, которые не удалось распознать (остается около 3500 фото), запись *размера* лица и бинарного значения яркости *пикселей* в 68 зонах вокруг ключевых точек в их черно-белой версии (размер зон - 8-17% от ширины лица, сейчас подбирается) в dataset (5 х 0.2-0.5 GB)
> псевдокод: ```image.to_greyscale().cut([zone_schape])```

- notebook.ipynb (до метки "Legacy"):
1. Добавление гистограмм распределения яркости к склейке датасетов
2. Добавление метрик std и noise для каждой зоны
3. Очистка от исходных массивов пикселей, запись на диск (30 MB)

- notebook2.ipynb:
1. Отсекание лишних фейков из исходного датасета (остается 1000=500*2 из 3500 фото) для избавления от перекоса (по крайней мере в процессе подбора методов, такой датасет удобнее, иначе недо/переобучение вощникает слишком часто)
2. Сбор std и avg значений для каждого из классов => определение точек, с наибольшим значением ```|avg_fake-avg_real| / (std_fake + std_real)```
> sklearn.feature_selection.SelectKBest работает плохо, т.к. не включает метрики для всего лица, которые нужны для нормализации значений по точкам

3. Выборка самых показательных features + features по всему лицу
4. Обучение моделей

> Так как 5 из 8 моделей дают сопоставимую точность, на данном этапе целесообразно работать надо подбором фичей и размерами сканируемых зон, мб фильтрацией плохих/маленьких фото, тюнинг моделей (тщательный) следует проводть позднее

## Потенциальные темы НИРов

> 1 тема != 1 НИР, темы могут быть объединены или разбиты

- процесс сборки данных: введение про проблему и методы анализа дипфейков, датасет DFDC, распознавание точек, OpenCV, DLib, датасет 300-W
- Предобработка данных: фильтрация, гистограммы и агрегационные метрики
- Подбор features, метрики метрик, метрики по всему лицу
- Выбор и тюнинг моделей, влияние размера и перекоса датасета и т.д.

На более поздних этапах (опционально):
- выстраивание вокруг модели эконсистемы: веб-приложение, комплект клиентов на React-native / чат-бот, Google OAuth, Metamask Auth
- Биллинг для приложений, ценообразование, токены (можно крипту из тестнета (Goerli, Ether testnet) ради забавы прикрутить), мониторинг (Grafana?), Docker-Kuber, балансировка и прочие умные штуки

# Обработка данных
<hr>

> 990 фото, баланс классов 1:1, размер зон - 8..17% от ширины лица (в процессе экспериментов), f1-метрика на разных моделях - 0.60..0.70

## Импорт dataset-а и агрегация данных для классов 'fake' и 'real'

In [61]:
import os
os.getcwd()

'C:\\Users\\sergey.astakhov\\Desktop\\BmstuDeepFake'

In [62]:
import pandas as pd
import numpy as np

In [63]:
df = pd.read_json("../deep_fake_src/dfdc_dataframes/df_total_0_4_compact_frame_10.json")
# df = pd.read_json("../deep_fake_src/dfdc_dataframes/dataframe_compact_total_27_frame_10_lim100_offset0.json")

In [64]:
# Балансировка классов 1:1
# Инверсные метрики для полиномов

# df = df.drop(columns=['index','face_size_px'])
df = pd.concat([
    df[df.fake==True].sample(
        int(df[df.fake==False].shape[0]*1)), 
    df[df.fake==False]
])

face = df.filter(like='face', axis=1).copy()

for col in face:
    name = str(col) + '_reversed'
    df[name] = face[str(col)].map(lambda x: 1.0 / max(1,x))

df = df.filter(regex='^(.(?!(var)))*$', axis=1).filter(regex='^(.(?!(noise_1)))*$', axis=1)

print(df.shape)
# df = df.reset_index()
df.head()

(990, 281)


,index,filename,fake,face_size_px,pt_48_std,pt_49_std,pt_50_std,pt_51_std,pt_52_std,pt_53_std,...,pt_13_hist_noise_3,pt_14_hist_noise_3,pt_15_hist_noise_3,pt_16_hist_noise_3,overall_face_hist_noise_3,face_size_px_reversed,overall_face_std_reversed,overall_face_hist_std_reversed,overall_face_hist_noise_0.5_reversed,overall_face_hist_noise_3_reversed
61,62,mxibogtmey.mp4,True,99856,14.628458,21.289549,18.010666,21.224335,17.372715,34.847921,...,31.122449,23.979592,18.367347,42.857143,0.221319,0.000010,0.021351,0.002005,1.0,1.0
1529,733,rbtvfrxtou.mp4,True,88804,16.064051,33.455053,24.847009,21.386088,17.541314,12.985771,...,19.387755,4.591837,6.122449,7.142857,0.277015,0.000011,0.021765,0.002559,1.0,1.0
928,132,iafvtrwdna.mp4,True,126736,22.888016,20.555433,24.013741,27.744368,24.423468,29.732919,...,22.656250,12.500000,5.859375,33.593750,0.154652,0.000008,0.019386,0.002176,1.0,1.0
1714,918,rvvpazsffd.mp4,True,153664,6.799844,9.743831,9.948632,6.363006,6.742063,4.903990,...,0.000000,0.000000,0.925926,8.024691,0.143820,0.000007,0.024071,0.001252,1.0,1.0
3329,711,ejilmxtdpq.mp4,True,232324,5.732423,5.980013,7.038142,8.639158,11.319087,11.783769,...,2.083333,9.375000,12.673611,7.986111,0.073174,0.000004,0.038062,0.000490,1.0,1.0


In [65]:
# агрегация для фейков

df_fakes_compact = df[df.fake==True].filter(regex='^(.(?!(raw)))*$', axis=1) \
                                    .filter(regex='^(.(?!(ake)))*$', axis=1) \
                                    .filter(regex='^(.(?!(ilename)))*$', axis=1) \
                                    .filter(regex='^(.(?!(ist_simple)))*$', axis=1)
                                    
df_fakes_reduced = pd.DataFrame(df_fakes_compact.mean()).T
df_fakes_reduced['fake'] = True
df_fakes_reduced = df_fakes_reduced.set_index('fake')
df_fakes_reduced.filter(like='face', axis=1)

,face_size_px,overall_face_std,overall_face_hist_std,overall_face_hist_noise_0.5,overall_face_hist_noise_3,face_size_px_reversed,overall_face_std_reversed,overall_face_hist_std_reversed,overall_face_hist_noise_0.5_reversed,overall_face_hist_noise_3_reversed
fake,,,,,,,,,,
True,98025.608081,46.143719,511.792248,0.172276,0.288978,0.000015,0.023969,0.003232,0.998208,0.99584


In [66]:
# агрегация для реальных фото

df_real_compact = df[df.fake==False].filter(regex='^(.(?!(raw)))*$', axis=1) \
                                    .filter(regex='^(.(?!(ake)))*$', axis=1) \
                                    .filter(regex='^(.(?!(ilename)))*$', axis=1) \
                                    .filter(regex='^(.(?!(ist_simple)))*$', axis=1)
                                    
df_real_reduced = pd.DataFrame(df_real_compact.mean()).T
df_real_reduced['fake'] = False
df_real_reduced = df_real_reduced.set_index('fake')
df_real_reduced.filter(like='face', axis=1)

,face_size_px,overall_face_std,overall_face_hist_std,overall_face_hist_noise_0.5,overall_face_hist_noise_3,face_size_px_reversed,overall_face_std_reversed,overall_face_hist_std_reversed,overall_face_hist_noise_0.5_reversed,overall_face_hist_noise_3_reversed
fake,,,,,,,,,,
False,92126.343434,49.182392,534.771606,0.200743,0.308146,0.000016,0.022526,0.003057,0.999456,0.998436


In [67]:
# самые ярко-различающиеся по классам нfeatures

df_compare = pd.concat([df_real_reduced, df_fakes_reduced]).T
df_compare['diff_rel'] = abs(df_compare[False] - df_compare[True]) / (df_compare[False] + df_compare[True])
df_compare['diff'] = abs(df_compare[False] - df_compare[True]) 

df_compare = df_compare.sort_values(by=['diff_rel'], ascending=False)
df_compare.head(15)

fake,False,True,diff_rel,diff
pt_24_hist_noise_0.5,1.649462,1.134238,0.185086,0.515224
pt_19_hist_noise_0.5,1.641023,1.153223,0.174573,0.487801
pt_57_hist_noise_0.5,0.871193,0.627449,0.162643,0.243744
pt_30_hist_noise_0.5,1.692450,1.229350,0.158498,0.463100
pt_58_hist_noise_0.5,0.914622,0.681107,0.146338,0.233516
pt_57_hist_noise_3,16.263845,12.113761,0.146245,4.150084
pt_21_hist_noise_0.5,1.300257,0.974685,0.143112,0.325572
pt_56_hist_noise_0.5,0.956043,0.719896,0.140904,0.236147
pt_39_std,19.970445,15.340786,0.131110,4.629658
index,541.650505,420.167677,0.126305,121.482828


In [68]:
# самые стабильные features

df_fakes_std = pd.DataFrame(df_fakes_compact.std()).T
df_fakes_std['fake'] = True
df_fakes_std = df_fakes_std.set_index('fake')

df_real_std = pd.DataFrame(df_real_compact.std()).T
df_real_std['fake'] = False
df_real_std = df_real_std.set_index('fake')

df_std = pd.concat([df_real_reduced, df_fakes_reduced]).T
df_std['total_std'] = (df_std[False] + df_std[True])

df_std = df_std.sort_values(by=['total_std'], ascending=True)
df_std.head(10)

fake,False,True,total_std
face_size_px_reversed,0.000016,0.000015,0.000031
overall_face_hist_std_reversed,0.003057,0.003232,0.006289
overall_face_std_reversed,0.022526,0.023969,0.046495
overall_face_hist_noise_0.5,0.200743,0.172276,0.373019
overall_face_hist_noise_3,0.308146,0.288978,0.597124
pt_8_hist_noise_0.5,0.507715,0.476355,0.984070
pt_7_hist_noise_0.5,0.540935,0.526162,1.067097
pt_9_hist_noise_0.5,0.687613,0.566721,1.254335
pt_10_hist_noise_0.5,0.715615,0.592673,1.308288
pt_57_hist_noise_0.5,0.871193,0.627449,1.498642


In [69]:
# самые надежные (разилчие_по_классам / разброс) features
# (var не кажется действительно надежной, скорее всего просто зависит от std квадратично или как-то еще)

df_smart = pd.concat([pd.DataFrame(df_compare['diff']),pd.DataFrame(df_std['total_std'])], axis=1)
df_smart['metric'] = df_smart['diff'] / df_smart['total_std']
df_smart = df_smart.sort_values(by=['metric'], ascending=False)
df_smart.head(30)

,diff,total_std,metric
pt_24_hist_noise_0.5,0.515224,2.783700,0.185086
pt_19_hist_noise_0.5,0.487801,2.794246,0.174573
pt_57_hist_noise_0.5,0.243744,1.498642,0.162643
pt_30_hist_noise_0.5,0.463100,2.921801,0.158498
pt_58_hist_noise_0.5,0.233516,1.595729,0.146338
pt_57_hist_noise_3,4.150084,28.377606,0.146245
pt_21_hist_noise_0.5,0.325572,2.274942,0.143112
pt_56_hist_noise_0.5,0.236147,1.675939,0.140904
pt_39_std,4.629658,35.311231,0.131110
index,121.482828,961.818182,0.126305


## Подбор features [to be modified]

> опирается на интуитивные предположения + метод определения фейков по шуму

In [70]:
features = list(set(
    list(df_smart.filter(like='face_hist', axis=0).index) +  # for relative metrics
    list(df_smart.filter(like='noise_0.5', axis=0).head(7).index) +
    list(df_smart.filter(like='noise_3', axis=0).head(7).index) +
    list(df_smart.filter(like='std', axis=0).head(5).index) +
    list(df_smart.filter(like='reversed', axis=0).index) 
    # list(df_smart.filter(like='var', axis=0).head(3).index) 
))

print(len(features))
features

27


['overall_face_hist_std_reversed',
 'overall_face_hist_noise_0.5',
 'pt_56_hist_noise_3',
 'pt_1_std',
 'face_size_px_reversed',
 'overall_face_hist_std',
 'pt_56_hist_noise_0.5',
 'pt_41_std',
 'pt_7_hist_noise_3',
 'overall_face_hist_noise_0.5_reversed',
 'pt_30_hist_noise_0.5',
 'pt_57_hist_noise_0.5',
 'pt_24_hist_noise_0.5',
 'pt_58_hist_noise_3',
 'pt_36_std',
 'overall_face_std_reversed',
 'pt_39_std',
 'pt_57_std',
 'pt_8_hist_noise_3',
 'pt_57_hist_noise_3',
 'pt_21_hist_noise_0.5',
 'pt_58_hist_noise_0.5',
 'pt_19_hist_noise_0.5',
 'overall_face_hist_noise_3',
 'pt_45_hist_noise_3',
 'overall_face_hist_noise_3_reversed',
 'pt_39_hist_noise_3']

## Разбивка данных на train и test сплиты

In [71]:
# очистка dataset-а от лишних колонок перед тренировкой модели (необязательно?)

df_compact = df.filter(regex='^(.(?!(raw)))*$', axis=1) \
                                    .filter(regex='^(.(?!(ilename)))*$', axis=1) \
                                    .filter(regex='^(.(?!(ist_simple)))*$', axis=1)
df_compact.head(5)

,index,fake,face_size_px,pt_48_std,pt_49_std,pt_50_std,pt_51_std,pt_52_std,pt_53_std,pt_54_std,...,pt_13_hist_noise_3,pt_14_hist_noise_3,pt_15_hist_noise_3,pt_16_hist_noise_3,overall_face_hist_noise_3,face_size_px_reversed,overall_face_std_reversed,overall_face_hist_std_reversed,overall_face_hist_noise_0.5_reversed,overall_face_hist_noise_3_reversed
61,62,True,99856,14.628458,21.289549,18.010666,21.224335,17.372715,34.847921,26.254864,...,31.122449,23.979592,18.367347,42.857143,0.221319,0.000010,0.021351,0.002005,1.0,1.0
1529,733,True,88804,16.064051,33.455053,24.847009,21.386088,17.541314,12.985771,15.127192,...,19.387755,4.591837,6.122449,7.142857,0.277015,0.000011,0.021765,0.002559,1.0,1.0
928,132,True,126736,22.888016,20.555433,24.013741,27.744368,24.423468,29.732919,18.012355,...,22.656250,12.500000,5.859375,33.593750,0.154652,0.000008,0.019386,0.002176,1.0,1.0
1714,918,True,153664,6.799844,9.743831,9.948632,6.363006,6.742063,4.903990,2.099265,...,0.000000,0.000000,0.925926,8.024691,0.143820,0.000007,0.024071,0.001252,1.0,1.0
3329,711,True,232324,5.732423,5.980013,7.038142,8.639158,11.319087,11.783769,13.967545,...,2.083333,9.375000,12.673611,7.986111,0.073174,0.000004,0.038062,0.000490,1.0,1.0


In [72]:
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

In [73]:
y = df_compact['fake']
X = pd.DataFrame(df_compact.drop(columns=['fake']))
# selector = SelectKBest(f_classif, k=7)
# X = selector.fit_transform(X, y)
X =  df_compact[features]

# mask = selector.get_support()
# X = X[X.columns[mask]]

# df_compact.filter(like='size', axis=0).head()

# print(df_compact[X.columns[mask]].shape)
# df_compact[X.columns[mask]].head()

pd.DataFrame(X).head()

,overall_face_hist_std_reversed,overall_face_hist_noise_0.5,pt_56_hist_noise_3,pt_1_std,face_size_px_reversed,overall_face_hist_std,pt_56_hist_noise_0.5,pt_41_std,pt_7_hist_noise_3,overall_face_hist_noise_0.5_reversed,...,pt_57_std,pt_8_hist_noise_3,pt_57_hist_noise_3,pt_21_hist_noise_0.5,pt_58_hist_noise_0.5,pt_19_hist_noise_0.5,overall_face_hist_noise_3,pt_45_hist_noise_3,overall_face_hist_noise_3_reversed,pt_39_hist_noise_3
61,0.002005,0.159229,22.448980,18.179157,0.000010,498.826134,0.000000,24.652142,2.551020,1.0,...,13.365727,5.612245,14.795918,0.000000,0.000000,0.000000,0.221319,27.551020,1.0,29.591837
1529,0.002559,0.181298,26.020408,14.162092,0.000011,390.833509,0.000000,27.609867,18.877551,1.0,...,21.094413,13.265306,20.408163,0.000000,0.000000,0.000000,0.277015,17.346939,1.0,19.897959
928,0.002176,0.071803,14.843750,20.639730,0.000008,459.456434,1.953125,22.595581,8.984375,1.0,...,10.557338,9.765625,7.812500,2.343750,3.515625,1.953125,0.154652,28.906250,1.0,23.437500
1714,0.001252,0.092409,0.925926,15.471388,0.000007,798.859358,0.000000,13.843653,1.234568,1.0,...,2.395952,2.777778,1.543210,3.086420,0.617284,0.617284,0.143820,4.629630,1.0,7.407407
3329,0.000490,0.048639,6.423611,7.347330,0.000004,2041.053003,0.520833,11.645459,1.041667,1.0,...,9.486299,1.215278,4.687500,1.041667,0.347222,1.041667,0.073174,15.798611,1.0,3.993056


In [74]:
# print(df_compact.filter(like='size', axis=1).shape)
# df_compact.filter(like='size', axis=1).head()

In [75]:
# sizes_col = X.filter(like='size', axis=1).copy()
# X = pd.concat([
#     df_compact[X.columns[mask]],
#     sizes_col
# ], axis=0)
# # X.join(df_compact.filter(like='size', axis=1), lsuffix='_caller', rsuffix='_other')
# X.head()

In [76]:
X_train, X_test, y_train, y_test = train_test_split( 
    X, 
    y,
    test_size=0.20, 
    random_state=420)

In [77]:
pd.DataFrame(X_train).head()

,overall_face_hist_std_reversed,overall_face_hist_noise_0.5,pt_56_hist_noise_3,pt_1_std,face_size_px_reversed,overall_face_hist_std,pt_56_hist_noise_0.5,pt_41_std,pt_7_hist_noise_3,overall_face_hist_noise_0.5_reversed,...,pt_57_std,pt_8_hist_noise_3,pt_57_hist_noise_3,pt_21_hist_noise_0.5,pt_58_hist_noise_0.5,pt_19_hist_noise_0.5,overall_face_hist_noise_3,pt_45_hist_noise_3,overall_face_hist_noise_3_reversed,pt_39_hist_noise_3
1874,0.000908,0.059993,3.000000,16.133186,0.000005,1101.052972,0.250000,28.313497,5.250000,1.0,...,6.773839,5.250000,3.000000,1.750000,1.250000,3.25000,0.100873,16.50000,1.0,13.00000
2789,0.009024,0.323947,29.687500,40.762601,0.000026,110.809503,0.000000,54.032750,31.250000,1.0,...,30.158335,15.625000,45.312500,0.000000,0.000000,0.00000,0.584124,68.75000,1.0,40.62500
1367,0.006143,0.316885,25.000000,32.988390,0.000020,162.799532,0.000000,45.425626,36.000000,1.0,...,13.803112,16.000000,31.000000,0.000000,0.000000,0.00000,0.470344,42.00000,1.0,41.00000
628,0.001846,0.116083,16.326531,3.777757,0.000010,541.801473,0.000000,15.403128,29.591837,1.0,...,10.239666,22.959184,9.183673,0.000000,0.000000,0.00000,0.192102,2.55102,1.0,27.55102
2329,0.002955,0.152439,12.890625,28.787787,0.000009,338.385216,3.515625,36.606967,29.296875,1.0,...,12.582340,4.687500,13.281250,1.171875,5.859375,7.03125,0.223081,12.50000,1.0,26.56250


In [78]:
y_train.head()

1874    False
2789     True
1367     True
628      True
2329    False
Name: fake, dtype: bool

## Обучение моделей

### Служебные функции

In [79]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import SGDClassifier


In [80]:
import joblib

In [81]:
def save_model(model, X_train, y_train, X_test, y_test, p=2, name='my_model.pkl'):
    clf = make_pipeline(
        StandardScaler(),
        PolynomialFeatures(degree=p), 
        model
    )
    clf.fit(X_train, y_train)
    joblib.dump(clf, name)

In [82]:
def train_polynomial_pipeline(model, X_train, y_train, X_test, y_test, p=2):
    clf = make_pipeline(
        StandardScaler(),
        PolynomialFeatures(degree=p), 
        model
    )
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    return (
        confusion_matrix(y_test, y_pred), 
        classification_report(
            y_test, 
            y_pred, 
            target_names=['class "real"', 'class "fakes"'], 
            zero_division=np.nan)
    )

In [83]:
def print_polynomial_pipeline(model, X_train, y_train, X_test, y_test, p=2):
    confusion_matrix, classification_report = train_polynomial_pipeline(
        model, 
        X_train, 
        y_train, 
        X_test, 
        y_test,
        p
    )
    print(confusion_matrix)
    print(classification_report)

### SGDClassifier (f1=0.60) [to be modified]

In [84]:
print_polynomial_pipeline(
    SGDClassifier(),
    X_train, 
    y_train, 
    X_test, 
    y_test,
    p=2
) 

[[64 37]
 [36 61]]
               precision    recall  f1-score   support

 class "real"       0.64      0.63      0.64       101
class "fakes"       0.62      0.63      0.63        97

     accuracy                           0.63       198
    macro avg       0.63      0.63      0.63       198
 weighted avg       0.63      0.63      0.63       198



### SVC (f1=0.64) [to be modified]

In [85]:
print_polynomial_pipeline(
    SVC(gamma='auto'),
    X_train, 
    y_train, 
    X_test, 
    y_test,
    p=2
) 

[[46 55]
 [14 83]]
               precision    recall  f1-score   support

 class "real"       0.77      0.46      0.57       101
class "fakes"       0.60      0.86      0.71        97

     accuracy                           0.65       198
    macro avg       0.68      0.66      0.64       198
 weighted avg       0.69      0.65      0.64       198



### LogisticRegression (f1=0.61) [to be modified]

In [93]:
print_polynomial_pipeline(
    # LogisticRegression(max_iter=15000)
    LogisticRegression(
        max_iter=15000, 
        # penalty=None,
        class_weight='balanced',
        solver='liblinear',
        # tol=1e-6
    ),
    X_train, 
    y_train, 
    X_test, 
    y_test,
    p=2
) 
save_model(
    # LogisticRegression(max_iter=15000)
    LogisticRegression(
        max_iter=15000, 
        # penalty=None,
        class_weight='balanced',
        solver='liblinear',
        # tol=1e-6
    ),
    X_train, 
    y_train, 
    X_test, 
    y_test,
    p=2,
    name="logistic_regression.pkl"
) 

[[67 34]
 [27 70]]
               precision    recall  f1-score   support

 class "real"       0.71      0.66      0.69       101
class "fakes"       0.67      0.72      0.70        97

     accuracy                           0.69       198
    macro avg       0.69      0.69      0.69       198
 weighted avg       0.69      0.69      0.69       198



### NN - MLPClassifier (f1=0.66) [to be modified]

In [87]:
print_polynomial_pipeline(
    # MLPClassifier(max_iter=5000), # 0.57
    MLPClassifier(
        solver='lbfgs', 
        hidden_layer_sizes=(34,), # 5, 7, 7, 3 - 0.60
        random_state=1, 
        alpha=0.001, 
        # activation='relu',
        tol=1e-6,
        max_fun=15000,
        max_iter=15000),
    X_train, 
    y_train, 
    X_test, 
    y_test,
    p=1
) 

[[66 35]
 [36 61]]
               precision    recall  f1-score   support

 class "real"       0.65      0.65      0.65       101
class "fakes"       0.64      0.63      0.63        97

     accuracy                           0.64       198
    macro avg       0.64      0.64      0.64       198
 weighted avg       0.64      0.64      0.64       198



### RandomForestClassifier (good, f1=0.64) [to be modified] 

In [88]:
print_polynomial_pipeline(
    RandomForestClassifier(max_depth=7, random_state=42),
    X_train, 
    y_train, 
    X_test, 
    y_test,
    p=2
) 
save_model(
    RandomForestClassifier(max_depth=7, random_state=42),
    X_train, 
    y_train, 
    X_test, 
    y_test,
    p=2,
    name="random_forest_10.pkl"
) 

[[61 40]
 [30 67]]
               precision    recall  f1-score   support

 class "real"       0.67      0.60      0.64       101
class "fakes"       0.63      0.69      0.66        97

     accuracy                           0.65       198
    macro avg       0.65      0.65      0.65       198
 weighted avg       0.65      0.65      0.65       198



### DecisionTreeClassifier (f1=0.59) [to be modified]

In [89]:
print_polynomial_pipeline(
    DecisionTreeClassifier(max_depth=5, min_samples_split=10, min_samples_leaf=5),
    X_train, 
    y_train, 
    X_test, 
    y_test,
    p=2
)

[[46 55]
 [32 65]]
               precision    recall  f1-score   support

 class "real"       0.59      0.46      0.51       101
class "fakes"       0.54      0.67      0.60        97

     accuracy                           0.56       198
    macro avg       0.57      0.56      0.56       198
 weighted avg       0.57      0.56      0.56       198



### RadiusNeighborsClassifier (f1=0.46) [уязвим к выбросам]

In [90]:
print_polynomial_pipeline(
    RadiusNeighborsClassifier(
        radius=100, 
        weights='distance', 
        p=1, 
        outlier_label='most_frequent'),
    X_train, 
    y_train, 
    X_test, 
    y_test,
    p=2
)

[[28 73]
 [14 83]]
               precision    recall  f1-score   support

 class "real"       0.67      0.28      0.39       101
class "fakes"       0.53      0.86      0.66        97

     accuracy                           0.56       198
    macro avg       0.60      0.57      0.52       198
 weighted avg       0.60      0.56      0.52       198



### KNeighborsClassifier (f1=0.69) [to be modified]

In [92]:
print_polynomial_pipeline(
    KNeighborsClassifier(n_neighbors=6, weights='distance'),
    X_train, 
    y_train, 
    X_test, 
    y_test,
    p=1
)
save_model(
    KNeighborsClassifier(n_neighbors=6, weights='distance'),
    X_train, 
    y_train, 
    X_test, 
    y_test,
    p=1,
    name="knn.pkl"
) 

[[58 43]
 [28 69]]
               precision    recall  f1-score   support

 class "real"       0.67      0.57      0.62       101
class "fakes"       0.62      0.71      0.66        97

     accuracy                           0.64       198
    macro avg       0.65      0.64      0.64       198
 weighted avg       0.65      0.64      0.64       198

